In [48]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
port = 31700
shelter = AnimalShelter(username, password, port)

# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('FinalProject')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:Grazioso Salvare Logo/png;base64,{}'.format(encoded_image.decode()), width='2500px', height='250px')),
    html.Hr(),
    html.Div([
        dcc.RadioItems(['Water Rescue', 'Wilderness Rescue', 'Disaster Rescue', 'Reset'], 'Reset', id = 'radio-id')
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
        editable = False,
        column_selectable = False,
        row_deletable = False,
        row_selectable = 'single',
        sort_action = 'native',
        sort_mode = 'multi',
        page_action = 'native',
        page_current = 0,
        page_count = 10,
        page_size = 15,
        selected_rows = [0],
        selected_columns = []
    ),
    html.Br(),
    html.Hr(),
    html.Center(html.B(html.H1("Charles Beavers"))),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it.

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
        }for i in selected_columns]

# This callback will update the dashboard when a filter selection is made.
@app.callback(
    Output('datatable-id', "data"),
    [Input('radio-id', 'value')]
)
def on_selection(value):

    df = pd.DataFrame.from_records(shelter.read({}))
    
    if str(value) == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed': {'$in': ["Labrador Retriever Mix",
            "Chesapeake Bay Retriever", "Newfoundland"]}}))
    elif str(value) == 'Wilderness Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed': {'$in': ["German Shepherd", "Alaskan Malamute", 
            "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}))
    elif str(value) == 'Disaster Rescue':
        df = pd.DataFrame.from_records(shelter.read({'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 
            'Golden Retriever', 'Bloodhound', 'Rottweiler']}}))
    
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

#Callback updates graph with filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
])
def update_graphs(viewData):
    #Using histogram to display animal available by breed
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed', title='Availale Dogs by Breed')
        )    
    ]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], 
            zoom=10, 
            children=[dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for 
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([html.H1("Animal Name"),
                html.P(dff.iloc[row,9])])])])]

app.run_server(debug=True)

Animal Shelter
reading
Dash app running on http://127.0.0.1:13735/
reading
